## Process

- [ ] Load CEMS data and perform initial cleaning
    - [ ] Crosswalk the CAMD ids to EIA ids
    - [ ] Convert UTC timestamps to local time to assist with merging EIA data
- [ ] Determine the primary fuel type for each EPA unit 
- [ ] Determine boiler type for each EPA unit
- [ ] Fill missing CO2, NOx and SOx data using fuel type, boiler type, and heat input
    - [ ] For observations with heat input but no measured emissions, estimate emissions based on factors
    - [ ] Aggregate data to monthly level. For units that have no reported data for a specific month, check if data for that month was reported to EIA and merge in the heat input data if available. Use the monthly heat input data to estimate emissions.
    - [ ] test
    

### Updates to PUDL
 - Need to load boiler firing type data from EIA-860
 - In CEMS data package, change plant_id_eia to plant_id_camd

In [3]:
%load_ext autoreload
%autoreload 2

# Standard libraries
import logging
import sys
import os
import pathlib

# 3rd party libraries
import dask.dataframe as dd
from dask.distributed import Client
import numpy as np
import pandas as pd
import sqlalchemy as sa
import importlib

# Local libraries
import pudl
from pudl.analysis import egrid

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [2]:
pudl_settings = pudl.workspace.setup.get_defaults()
display(pudl_settings)

pudl_engine = sa.create_engine(pudl_settings['pudl_db'])
#display(pudl_engine)

year = 2018

{'pudl_in': 'C:\\Users\\Greg\\pudl_workspace',
 'data_dir': 'C:\\Users\\Greg\\pudl_workspace\\data',
 'settings_dir': 'C:\\Users\\Greg\\pudl_workspace\\settings',
 'pudl_out': 'C:\\Users\\Greg\\pudl_workspace',
 'sqlite_dir': 'C:\\Users\\Greg\\pudl_workspace\\sqlite',
 'parquet_dir': 'C:\\Users\\Greg\\pudl_workspace\\parquet',
 'datapkg_dir': 'C:\\Users\\Greg\\pudl_workspace\\datapkg',
 'notebook_dir': 'C:\\Users\\Greg\\pudl_workspace\\notebook',
 'ferc1_db': 'sqlite:///C:\\Users\\Greg\\pudl_workspace\\sqlite\\ferc1.sqlite',
 'pudl_db': 'sqlite:///C:\\Users\\Greg\\pudl_workspace\\sqlite\\pudl.sqlite'}

### Read CEMS data from parquet

Available columns in CEMS
\['plant_id_eia', 'unitid', 'operating_datetime_utc',
'operating_time_hours', 'gross_load_mw', 'steam_load_1000_lbs',
'so2_mass_lbs', 'so2_mass_measurement_code', 'nox_rate_lbs_mmbtu',
'nox_rate_measurement_code', 'nox_mass_lbs',
'nox_mass_measurement_code', 'co2_mass_tons',
'co2_mass_measurement_code', 'heat_content_mmbtu', 'facility_id',
'unit_id_epa', 'state'\]

In [33]:

epacems_path = pudl_settings['parquet_dir'] + f'/epacems/year={year}'

my_columns = ['plant_id_eia', 'unitid', 'operating_datetime_utc',
'operating_time_hours', 'gross_load_mw', 'steam_load_1000_lbs',
'so2_mass_lbs', 'nox_rate_lbs_mmbtu', 'nox_mass_lbs', 
'co2_mass_tons', 'heat_content_mmbtu']

cems_df = pd.read_parquet(epacems_path, columns=my_columns).query("operating_time_hours > 0")  # only load observations when the plant was operating

# rename cems plant_id_eia to plant_id_epa until the crosswalk integration is finished
cems_df = cems_df.rename(columns={'plant_id_eia': 'plant_id_camd', 'unitid':'unit_id_camd'})
cems_df.head(3)

,plant_id_camd,unit_id_camd,operating_datetime_utc,operating_time_hours,gross_load_mw,steam_load_1000_lbs,so2_mass_lbs,nox_rate_lbs_mmbtu,nox_mass_lbs,co2_mass_tons,heat_content_mmbtu,plant_id_eia
0,3,1,2018-01-01 06:00:00+00:00,1.0,27.0,NaN,4.250,0.082,31.350,22.650,382.399994,3.0
1,3,1,2018-01-01 07:00:00+00:00,1.0,29.0,NaN,4.402,0.083,32.987,23.562,397.399994,3.0
2,3,1,2018-01-01 08:00:00+00:00,1.0,32.0,NaN,4.664,0.083,35.902,25.652,432.399994,3.0


### Crosswalk the EPA plant and unit IDs to EIA plant and generator IDs

In [2]:
# load epa-eia crosswalk data
EPA_EIA_crosswalk = pd.read_csv("https://raw.githubusercontent.com/USEPA/camd-eia-crosswalk/master/camd_eia_crosswalk.csv", usecols=['CAMD_ORIS_CODE','CAMD_UNIT_ID','EIA_PLANT_CODE','EIA_GENERATOR_ID'])

# Because there can be multiple generators matched to a single boiler, we want to create a column that lists all of the generator IDs associated with the unit
generator_id_list = EPA_EIA_crosswalk.groupby(['CAMD_ORIS_CODE','CAMD_UNIT_ID'])['EIA_GENERATOR_ID'].apply(list)

EPA_EIA_crosswalk = EPA_EIA_crosswalk.groupby(['CAMD_ORIS_CODE','CAMD_UNIT_ID']).max()
EPA_EIA_crosswalk = EPA_EIA_crosswalk.merge(generator_id_list, how='left', left_index=True, right_index=True).reset_index()
EPA_EIA_crosswalk.head(6)

,CAMD_ORIS_CODE,CAMD_UNIT_ID,EIA_PLANT_CODE,EIA_GENERATOR_ID
0,3,1,3.0,[1]
1,3,2,3.0,[2]
2,3,4,3.0,[4]
3,3,5,3.0,[5]
4,3,6A,3.0,"[A1ST, A1CT]"
5,3,6B,3.0,"[A1CT2, A1ST]"


In [59]:

EPA_EIA_crosswalk = EPA_EIA_crosswalk.rename(columns={'CAMD_ORIS_CODE': 'plant_id_camd', 'EIA_PLANT_CODE':'plant_id_eia', 'CAMD_UNIT_ID':'unit_id_camd','EIA_GENERATOR_ID':'generator_id_eia'})
cems_df = cems_df.merge(EPA_EIA_crosswalk, how='left', on='plant_id_camd')

# if the merge resulted in any missing plant_id associations, fill with the epa plant_id
cems_df['plant_id_eia'] = cems_df['plant_id_eia'].fillna(
    cems_df['plant_id_camd'])

# EPA's eGRID documentation identifies several plants that are not grid-connected and should be removed
non_grid_connected_plant_ids = list(pd.read_csv(importlib.resources.open_text(
        'pudl.package_data.epa.egrid', 'table_4-2_plants_not_connected_to_grid.csv'),
        usecols=['Plant ID'])['Plant ID'])
cems_df = cems_df[~cems_df['plant_id_eia'].isin(non_grid_connected_plant_ids)]

#if any co2_mass_tons is reported as zero (while the plant was operating), change this to a missing value so that it can be calculated
cems_df['co2_mass_tons'] = cems_df['co2_mass_tons'].replace({0:np.nan})

cems_df.head(3)

KeyError: 'plant_id_eia'

### Add local datetime and report month
This will allow us to merge monthly data from the EIA reports

In [34]:
# ADD LOCAL TIMEZONE DATA

# load information about the time zone for each plant
datapkg_path = pudl_settings['datapkg_dir'] + '/to_parquet/epacems-eia/data/'

plant_timezones = pd.read_csv((datapkg_path + 'plants_entity_eia.csv'), usecols=['plant_id_eia','timezone'])

#merge the tz data into the cems data
cems_df = cems_df.merge(plant_timezones, how='left', on='plant_id_eia')

# create a report date column to enable matching with the EIA data
cems_df['report_month'] = np.NaN

for tz in list(cems_df['timezone'].unique()):
    tz_mask = cems_df['timezone'] == tz
    cems_df.loc[tz_mask, 'report_month'] = cems_df.loc[tz_mask, 'operating_datetime_utc'].dt.tz_convert(tz).dt.month
cems_df.head(3)

,plant_id_camd,unit_id_camd,operating_datetime_utc,operating_time_hours,gross_load_mw,steam_load_1000_lbs,so2_mass_lbs,nox_rate_lbs_mmbtu,nox_mass_lbs,co2_mass_tons,heat_content_mmbtu,plant_id_eia,timezone
0,3,1,2018-01-01 06:00:00+00:00,1.0,27.0,NaN,4.250,0.082,31.350,22.650,382.399994,3.0,America/Chicago
1,3,1,2018-01-01 07:00:00+00:00,1.0,29.0,NaN,4.402,0.083,32.987,23.562,397.399994,3.0,America/Chicago
2,3,1,2018-01-01 08:00:00+00:00,1.0,32.0,NaN,4.664,0.083,35.902,25.652,432.399994,3.0,America/Chicago


## Unit File
We want to aggregate the cems data by unit and month so that we can compare it to the EIA-923 data, which is aggregated at the month level. However, while we have the data at the hourly level, we first need to perform a couple of steps:  

1. If an observation contains heat input data but is missing CO2, NOx, or SO2 data, we can calculate a replacement value based on 

2. Since the datetime is in UTC, we need to convert the datetime back into the local tz before aggregating by month. 

### Assign a primary fuel type to each CAMD unit
We use reported fuel heat input data from EIA-923 to determine primary fuel type. We will create a table that matches each CAMD unit to a primary fuel type.
However, because there is not a direct 1:1 match between CAMD units and EIA generators/boilers, we use a sequential approach to assign fuel types:

1. First, we need to calculate the primary fuel type for each EIA boiler. 
2. Identify which plants have boilers that all use a single fuel type. We can assign this fuel type as the primary fuel to all units at this plant.
2. Next, if there is a 1:1 match between a CAMD unit and EIA boiler
3. 


Determining primary fuel by boiler  
1. First determine by boiler
2. Then match boiler to generators
3. Make sure generator number incorporated in crosswalk
4. Try to merge on boiler, then on generator


In [44]:
# Calculate primary fuel for each unit for each month based on eia-923 data
datapkg_path = pudl_settings['datapkg_dir'] + '/to_parquet/epacems-eia/data/'

boiler_fuel_eia923_df = pd.read_csv((datapkg_path + 'boiler_fuel_eia923.csv'), parse_dates=['report_date'])

#only keep data for the current year
boiler_fuel_eia923_df = boiler_fuel_eia923_df[boiler_fuel_eia923_df.report_date.dt.year == year]

#determine the primary fuel based on the fuel with the maximum mmbtu consumed
primary_fuel = egrid.determine_primary_fuel(bf_df=boiler_fuel_eia923_df, fuel_thresh=0.5, level='boiler')
primary_fuel['report_month'] = primary_fuel.report_date.dt.month
primary_fuel.head(3)

,plant_id_eia,boiler_id,report_date,primary_fuel,report_month
0,3,1,2018-01-01,NG,1
1,3,2,2018-01-01,NG,1
2,3,4,2018-01-01,BIT,1


### Calculate missing CO2 data
1. Merge primary fuel type information
2. Load emission factors for CO2
3. Calculate missing CO2 mass

In [45]:
#merge primary fuel data into cems
# TODO: Need to check if unit_id_camd needs to be crosswalked with the eia boiler_id
cems_df = cems_df.merge(primary_fuel, how='left', left_on=['plant_id_eia','unit_id_camd','report_month'], right_on=['plant_id_eia','boiler_id','report_month'])
cems_df.head(3)

,plant_id_camd,unit_id_camd,operating_datetime_utc,operating_time_hours,gross_load_mw,steam_load_1000_lbs,so2_mass_lbs,nox_rate_lbs_mmbtu,nox_mass_lbs,co2_mass_tons,heat_content_mmbtu,plant_id_eia,timezone,report_date_x,report_month,boiler_id,report_date_y,primary_fuel
0,3,1,2018-01-01 06:00:00+00:00,1.0,27.0,NaN,4.250,0.082,31.350,22.650,382.399994,3.0,America/Chicago,1.0,1.0,1,2018-01-01,NG
1,3,1,2018-01-01 07:00:00+00:00,1.0,29.0,NaN,4.402,0.083,32.987,23.562,397.399994,3.0,America/Chicago,1.0,1.0,1,2018-01-01,NG
2,3,1,2018-01-01 08:00:00+00:00,1.0,32.0,NaN,4.664,0.083,35.902,25.652,432.399994,3.0,America/Chicago,1.0,1.0,1,2018-01-01,NG


In [47]:
# load the CO2 emission factors and fill missing values
CO2_Emission_Factors = pd.read_csv(importlib.resources.open_binary(
        'pudl.package_data.epa.egrid', 'table_C1_emission_factors_for_CO2_CH4_N2O.csv'), usecols=['EIA Fuel Type Code','CO2 EF (ton CO2/mmBtu)'])

missing = cems_df[cems_df['co2_mass_tons'].isnull()]
print(f'There are {len(missing)} observations with missing CO2 values')

# add emission factor to missing df
missing = missing.merge(CO2_Emission_Factors, how='left',
                        left_on='primary_fuel', right_on='EIA Fuel Type Code')
# calculate missing co2 data
missing['co2_mass_tons'] = missing['heat_content_mmbtu'] * missing['CO2 EF (ton CO2/mmBtu)']

# update the main df with the missing data
#cems_df['co2_mass_tons'].update(missing['co2_mass_tons'])

#print(f'After calculating CO2 from heat rate, there are {len(cems_df[cems_df.co2_mass_tons.isnull()])} observations with missing CO2 values')

There are 1528334 observations with missing CO2 values
After calculating CO2 from heat rate, there are 1528282 observations with missing CO2 values


In [48]:
missing.head(5)

,plant_id_camd,unit_id_camd,operating_datetime_utc,operating_time_hours,gross_load_mw,steam_load_1000_lbs,so2_mass_lbs,nox_rate_lbs_mmbtu,nox_mass_lbs,co2_mass_tons,heat_content_mmbtu,plant_id_eia,timezone,report_date_x,report_month,boiler_id,report_date_y,primary_fuel,EIA Fuel Type Code,CO2 EF (ton CO2/mmBtu)
0,3,6B,2018-01-12 04:00:00+00:00,0.25,1.0,NaN,0.00,0.012,0.000000,0.00000,0.000000,3.0,America/Chicago,1.0,1.0,6B,2018-01-01,NG,NG,0.05844
1,26,2,2018-01-09 00:00:00+00:00,0.25,4.0,NaN,2.05,0.000,0.000000,0.02574,0.250000,26.0,America/Chicago,1.0,1.0,2,2018-01-01,BIT,BIT,0.10296
2,52140,Z006,2018-01-01 06:00:00+00:00,1.00,0.0,283.0,NaN,0.328,111.900002,NaN,341.500000,52140.0,America/Chicago,1.0,1.0,NaN,NaT,NaN,NaN,NaN
3,52140,Z006,2018-01-01 07:00:00+00:00,1.00,0.0,280.0,NaN,0.335,115.099998,NaN,344.000000,52140.0,America/Chicago,1.0,1.0,NaN,NaT,NaN,NaN,NaN
4,52140,Z006,2018-01-01 08:00:00+00:00,1.00,0.0,262.0,NaN,0.347,114.699997,NaN,330.899994,52140.0,America/Chicago,1.0,1.0,NaN,NaT,NaN,NaN,NaN


### Calculate missing NOx data
1. Merge information about prime mover and boiler firing type


In [ ]:
# CALCULATE MISSING NOX DATA

In [ ]:
# CALCULATE MISSING SO2 DATA

In [59]:
cems_df[cems_df['co2_mass_tons'].isnull()]

,plant_id_camd,unit_id_camd,operating_datetime_utc,operating_time_hours,gross_load_mw,steam_load_1000_lbs,so2_mass_lbs,nox_rate_lbs_mmbtu,nox_mass_lbs,co2_mass_tons,heat_content_mmbtu,unit_id_epa,plant_id_eia,timezone,operating_datetime_local
3004,3,6B,2018-01-12 04:00:00+00:00,0.25,1.0,NaN,0.00,0.012,0.000000,NaN,0.000000,7,3.0,America/Chicago,2018-01-11 22:00:00-06:00
9334,26,2,2018-01-09 00:00:00+00:00,0.25,4.0,NaN,2.05,0.000,0.000000,NaN,0.250000,31,26.0,America/Chicago,2018-01-08 18:00:00-06:00
27866,52140,Z006,2018-01-01 06:00:00+00:00,1.00,0.0,283.0,NaN,0.328,111.900002,NaN,341.500000,88248,52140.0,America/Chicago,2018-01-01 00:00:00-06:00
27867,52140,Z006,2018-01-01 07:00:00+00:00,1.00,0.0,280.0,NaN,0.335,115.099998,NaN,344.000000,88248,52140.0,America/Chicago,2018-01-01 01:00:00-06:00
27868,52140,Z006,2018-01-01 08:00:00+00:00,1.00,0.0,262.0,NaN,0.347,114.699997,NaN,330.899994,88248,52140.0,America/Chicago,2018-01-01 02:00:00-06:00
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
13940352,4158,BW43,2018-11-03 18:00:00+00:00,0.23,0.0,NaN,0.00,0.007,0.002000,NaN,0.230000,2635,4158.0,America/Denver,2018-11-03 12:00:00-06:00
13940353,4158,BW43,2018-11-05 14:00:00+00:00,0.07,0.0,NaN,0.00,0.000,0.000000,NaN,0.070000,2635,4158.0,America/Denver,2018-11-05 07:00:00-07:00
13940990,4158,BW44,2018-11-02 06:00:00+00:00,0.15,0.0,NaN,0.06,0.002,0.000000,NaN,0.150000,2636,4158.0,America/Denver,2018-11-02 00:00:00-06:00
13943698,6101,BW91,2018-11-29 02:00:00+00:00,0.33,0.0,NaN,0.00,0.000,0.000000,NaN,0.330000,2777,6101.0,America/Denver,2018-11-28 19:00:00-07:00
